# <div align="center">Predict Car Traffic Injury</div>

ggg

In this document we follow the CRISP-DM process and methodolgies...


# 1. CRISP-DM: Business Understanding

bla bla bla

<br>
<br>
<br>

# 2. CRISP-DM: Data Understanding

bla bla bla

## 2.1. Collect Initial Data


This a generic intial step, to start working on the data:<br>
2.1.1. Import the relevant Python packages that are going to be used.<br>
2.1.2. Acquire the data.<br>
2.1.3. Record of the data acquisition process.<br>

In [1]:
# 2.1.1. Import the relevant Python packages that are going to be used

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport



/home/moamen/anaconda3/envs/py311_pods_proj/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 2.1.2. Acquire the data

# A generic utilies file with generic functionallity
from misc.utilities import acquire_dataset

# Download & load the dataset
# If the file already downloaded, it will not be re-downloaded (to speed up work efficiency)
df = acquire_dataset()
# df

File already exists...


<span style="color:green">


2.1.3. Record of the data acquisition process.<br>

The data acquired from: https://catalog.data.gov/dataset/traffic-crashes-crashes
<br>
The above website belongs to USA government free datasets advised by the course instructions.

</span>


## 2.2. Data Description

In this phase we investigate the following aspects:<br>
2.2.1. Shape of the Dataset.<br>
2.2.2. Head snippet.<br>
2.2.3. Dataset info.<br>
2.2.4. Continuous Features Tabular Report.<br>
2.2.5. Categorical Features Tabular Report.<br>
2.2.6. Missing Values Summary.<br>
2.2.7. Correlation insights.<br>
2.2.8. HTML Reprot.<br>

In [4]:
# 2.2.1. Shape of the Dataset.

# Create a new DataFrame to display the shape information
shape_df = pd.DataFrame({
    'Aspect': ['Records (Instances)', 'Features (Columns)'],
    'Number': [df.shape[0], df.shape[1]]
})

# Display the DataFrame
shape_df


,Aspect,Number
0,Records (Instances),814788
1,Features (Columns),48


In [5]:
# 2.2.2. Head snippet
df.head()


,CRASH_RECORD_ID,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,NaN,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,OTHER,...,1.0,0.0,1.0,0.0,12,6,8,NaN,NaN,NaN
1,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,NaN,07/29/2023 02:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,1.0,0.0,14,7,7,41.854120,-87.665902,POINT (-87.665902342962 41.854120262952)
2,61fcb8c1eb522a6469b460e2134df3d15f82e81fd93e9c...,NaN,08/18/2023 05:58:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDALCYCLIST,NOT DIVIDED,...,1.0,0.0,1.0,0.0,17,6,8,41.942976,-87.761883,POINT (-87.761883496974 41.942975745006)
3,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,NaN,11/26/2019 08:38:00 AM,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDESTRIAN,ONE-WAY,...,0.0,0.0,1.0,0.0,8,3,11,NaN,NaN,NaN
4,a1d5f0ea90897745365a4cbb06cc60329a120d89753fac...,NaN,08/18/2023 10:45:00 AM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,OTHER,...,0.0,0.0,1.0,0.0,10,6,8,NaN,NaN,NaN


In [6]:
# 2.2.3. Dataset info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814788 entries, 0 to 814787
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                814788 non-null  object 
 1   CRASH_DATE_EST_I               60921 non-null   object 
 2   CRASH_DATE                     814788 non-null  object 
 3   POSTED_SPEED_LIMIT             814788 non-null  int64  
 4   TRAFFIC_CONTROL_DEVICE         814788 non-null  object 
 5   DEVICE_CONDITION               814788 non-null  object 
 6   WEATHER_CONDITION              814788 non-null  object 
 7   LIGHTING_CONDITION             814788 non-null  object 
 8   FIRST_CRASH_TYPE               814788 non-null  object 
 9   TRAFFICWAY_TYPE                814788 non-null  object 
 10  LANE_CNT                       199008 non-null  float64
 11  ALIGNMENT                      814788 non-null  object 
 12  ROADWAY_SURFACE_COND          

In [8]:
# 2.2.4. Continuous Features Tabular Report
continuous_features_tabular_report_df = df.describe(include=['number'])
continuous_features_transpose_tabular_report_df = continuous_features_tabular_report_df.transpose()

continuous_features_transpose_tabular_report_df


,count,mean,std,min,25%,50%,75%,max
POSTED_SPEED_LIMIT,814788.0,28.405658,6.168020,0.000000,30.000000,30.000000,30.000000,9.900000e+01
LANE_CNT,199008.0,13.330213,2961.608593,0.000000,2.000000,2.000000,4.000000,1.191625e+06
STREET_NO,814788.0,3689.471208,2886.846405,0.000000,1250.000000,3201.000000,5600.000000,4.511000e+05
BEAT_OF_OCCURRENCE,814783.0,1243.649172,705.298265,111.000000,714.000000,1211.000000,1822.000000,6.100000e+03
NUM_UNITS,814788.0,2.034870,0.452707,1.000000,2.000000,2.000000,2.000000,1.800000e+01
INJURIES_TOTAL,813012.0,0.189815,0.565798,0.000000,0.000000,0.000000,0.000000,2.100000e+01
INJURIES_FATAL,813012.0,0.001191,0.037361,0.000000,0.000000,0.000000,0.000000,4.000000e+00
INJURIES_INCAPACITATING,813012.0,0.019938,0.165285,0.000000,0.000000,0.000000,0.000000,1.000000e+01
INJURIES_NON_INCAPACITATING,813012.0,0.107078,0.422087,0.000000,0.000000,0.000000,0.000000,2.100000e+01
INJURIES_REPORTED_NOT_EVIDENT,813012.0,0.061608,0.319045,0.000000,0.000000,0.000000,0.000000,1.500000e+01


In [9]:
# 2.2.5. Categorical Features Tabular Report
categorical_features_tabular_report_df = df.describe(exclude=['number'])
categorical_features_transpose_tabular_report_df = categorical_features_tabular_report_df.transpose()
categorical_features_transpose_tabular_report_df


,count,unique,top,freq
CRASH_RECORD_ID,814788,814788,89dc61af34d393db950397f0cc06d53b56d1f5e5fa14d4...,1
CRASH_DATE_EST_I,60921,2,Y,53081
CRASH_DATE,814788,534899,12/29/2020 05:00:00 PM,30
TRAFFIC_CONTROL_DEVICE,814788,19,NO CONTROLS,463168
DEVICE_CONDITION,814788,8,NO CONTROLS,468583
WEATHER_CONDITION,814788,12,CLEAR,638955
LIGHTING_CONDITION,814788,6,DAYLIGHT,521065
FIRST_CRASH_TYPE,814788,18,PARKED MOTOR VEHICLE,189339
TRAFFICWAY_TYPE,814788,20,NOT DIVIDED,353897
ALIGNMENT,814788,6,STRAIGHT AND LEVEL,795005


In [ ]:
# 2.2.7. Correlation insights

# # Get the relevant columns
subset_df = df[continuous_features_tabular_report_df.columns]

# Calculate the correlation matrix
correlation_matrix = subset_df.corr()

# Create a heatmap using seaborn
plt.figure(figsize=(16, 14))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.8)

In [ ]:
# 2.2.8. Pandas-HTML Profiling Reprot.

# profile = ProfileReport(df, title="Pandas Profiling Report",explorative=True)
# profile.to_file("pandas_report.html")

# Disable this code in production code - it requires a strong PC to be to run it
# or to run it with smaller / configured set of profiling - please let me know if to share the report.


In [ ]:


IRREGULAR_CARD_SINGLE = 1
unique_counts = df.nunique()
# for feature_name, count in unique_counts.items():
#     if count == 1:
#         print(f"The feature '{feature_name}' has a unique count of 1 - Irregular Cardinality Issue")
unique_counts_single_threshold = unique_counts[unique_counts == IRREGULAR_CARD_SINGLE]
pd.DataFrame(unique_counts_single_threshold)

## 2.3. Explore the Data

1. Basic Statistics - For numeric models, to calculate Mean, Median, mode, ... <br>
2. Tabular Report - For Continuos and Categorical Features (Refer to HTML Reports please). <br>
3. Correlations + Heat Map


## 2.4. Verify Data Quality

2.4.1. Completnesss - Missing Values Summary.<br>
2.4.2. Irregular cardinality, could be one of the following cases:<br>
* Features with a cardinality of 1.
* Too high cardinality for categorical features.
* Too low cardinality for continous features.<br>

2.4.3. Consistency - Handle outliers, out of range data or invalid formats (if any)<br>
2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data.<br>



In [ ]:

# Check for missing values in each column
missing_values = df.isnull().sum()

# Check for missing values in the whole DataFrame
total_missing = missing_values.sum()
missing_values_sorted = missing_values.sort_values(ascending=False)

# Check the percentage of missing values for each column
missing_percentage_sorted = (missing_values_sorted / len(df)) * 100

# Create a DataFrame to summarize the completeness
completeness_report = pd.DataFrame({
    'Missing Values': missing_values_sorted,
    'Missing Percentage': missing_percentage_sorted
})

completeness_report

In [15]:
# 2.4.2. Irregular cardinality - Features with a cardinality of 1
IRREGULAR_CARD_SINGLE = 1

unique_counts = df.nunique()
# for feature_name, count in unique_counts.items():
#     if count == 1:
#         print(f"The feature '{feature_name}' has a unique count of 1 - Irregular Cardinality Issue")
unique_counts_single_threshold = unique_counts[unique_counts == IRREGULAR_CARD_SINGLE]
pd.DataFrame(unique_counts_single_threshold)

,0
INJURIES_UNKNOWN,1


In [16]:
# 2.4.2. Irregular cardinality - Too high cardinality for categorical features

IRREGULAR_CARD_MAX_THRESHOLD = 10

def get_unique_counts_exceed_max_threshold(df):
    cardinality = df.select_dtypes(exclude=['number']).apply(lambda x: x.nunique())
    high_cardinality_cols = cardinality[cardinality > IRREGULAR_CARD_MAX_THRESHOLD].index.tolist()
    # print("High Cardinality Columns:", high_cardinality_cols)
    return high_cardinality_cols

high_cardinality_cols = get_unique_counts_exceed_max_threshold(df)
# Categorical Features with carinality higher than IRREGULAR_CARD_MAX_THRESHOLD
print(f"Categorical Features with carinality higher than {IRREGULAR_CARD_MAX_THRESHOLD} cardinality are: {high_cardinality_cols}")

Categorical Features with carinality higher than 10 cardinality are: ['CRASH_RECORD_ID', 'CRASH_DATE', 'TRAFFIC_CONTROL_DEVICE', 'WEATHER_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE', 'DATE_POLICE_NOTIFIED', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NAME', 'LOCATION']


In [17]:
# 2.4.2. Irregular cardinality - Too low cardinality for continous features
IRREGULAR_CARD_MIN_THRESHOLD = 5
unique_counts = df.nunique()
unique_counts_below_min_threshold = unique_counts[unique_counts < IRREGULAR_CARD_MIN_THRESHOLD]
pd.DataFrame(unique_counts_below_min_threshold)


,0
CRASH_DATE_EST_I,2
REPORT_TYPE,3
CRASH_TYPE,2
INTERSECTION_RELATED_I,2
NOT_RIGHT_OF_WAY_I,2
HIT_AND_RUN_I,2
DAMAGE,3
STREET_DIRECTION,4
PHOTOS_TAKEN_I,2
STATEMENTS_TAKEN_I,2


<span style="color:cyan">

##### 2.4.2.  - Irregular Cardinality
##### Analysis

To do - describe the results from above.


</span>

In [ ]:
# 2.4.3. Consistency - Handle outliers, out of range data or invalid formats (if any)

## FIX ME


# Outliers
# def outliers(df, ft):
#     Q1 = df[ft].quantile(0.25)
#     Q3 = df[ft].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     ls = df.index[ (df[ft] < lower_bound) | (df[ft] > upper_bound) ]
#     return ls

# index_list = []
# for feature in df.columns:
#     if pd.api.types.is_numeric_dtype(df[feature]):
#         index_list.extend(outliers(continuous_features_df, feature))

# # def remove(df, ls):
# #     ls = sorted(set(ls))
# #     df = df.drop(ls)
# #     return df

# print(f"There are {len(set(index_list))} outliers")
# print(f"DataFrame Shape Before Removing Outliers {df.shape}")
# df_cleaned_after_outliers = remove(df_prepared, index_list)
# print(f"DataFrame Shape After  Removing Outliers {df_cleaned_after_outliers.shape}")

In [ ]:
# 2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data

def consistency_check_duplicated_instances(df):
    duplicate_rows = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicate_rows}")
    # If you want to actually see the duplicate rows, you can use:
    if duplicate_rows > 0:
        print(df[df.duplicated(keep=False)].sort_values(by=df.columns.tolist()))

consistency_check_duplicated_instances(df)


<br>
<br>
<br>

# 3. CRISP-DM: Data Preparation

3.1. Missing Values - Drop or Imputation.<br>
3.2. Irregular Cardinality.<br>
3.3. Outliers - To remove instances.<br>
3.4. Consistency - Invalid formats / Ranges.<br>

More details in each section.

In [20]:
# 3.1. Missing Values - Drop or Imputation

# Set the threshold for dropping columns
MISSING_THRESHOLD = 30.0

# Identify columns that have missing value percentage greater than the threshold
columns_to_drop = completeness_report[completeness_report['Missing Percentage'] >= MISSING_THRESHOLD].index

# Drop these columns from the DataFrame
df_after_drop_missing = df.drop(columns=columns_to_drop)

# Now, df has the columns dropped where the missing value percentage was higher than 30%
print("Features with missing instances higher than 30% that has been dropped")
columns_to_drop

Features with missing instances higher than 30% that has been dropped


Index(['WORKERS_PRESENT_I', 'DOORING_I', 'WORK_ZONE_TYPE', 'WORK_ZONE_I',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'NOT_RIGHT_OF_WAY_I',
       'CRASH_DATE_EST_I', 'INTERSECTION_RELATED_I', 'LANE_CNT',
       'HIT_AND_RUN_I'],
      dtype='object')

In [22]:
# 3.1. Missing Values -Imputation for values
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

# Your pandas code here

# Identify columns that have missing value percentage less than the imputation threshold
columns_to_impute = completeness_report[completeness_report['Missing Percentage'] < MISSING_THRESHOLD].index

df_after_imputation_missing = df_after_drop_missing.copy()

# Loop through the columns and perform imputation
for column in columns_to_impute:
    # if df_prepared[column].dtype == 'float64' or df[column].dtype == 'int64':
    if df_after_imputation_missing[column].dtype == 'numeric':
        # Impute numerical columns with the mean value
        df_after_imputation_missing[column].fillna(df[column].mean(), inplace=True)
    else:
        # Impute categorical columns with the mode value (the most frequent value)
        df_after_imputation_missing[column].fillna(df[column].mode()[0], inplace=True)


<span style="color:cyan">

##### 3.1. Missing Values -Imputation for values
##### Analysis

For missing values higher than 30%, we drop them.
For missing values lower than 30% - we do imputation. To be more accurate, in the terms of our dataset, missing values under 30% ranges between 0.1%-3% missing values so imputation is a very reasonable choice in this case.

Please refer to missing values report in the Data Understanding section.

</span>

In [26]:
# 3.3. Outliers - To remove instances.

df_clean_outliers = df_after_imputation_missing.copy()

# def outliers1(df, ft):
#     Q1 = df[ft].quantile(0.25)
#     Q3 = df[ft].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     ls = df.index[ (df[ft] < lower_bound) | (df[ft] > upper_bound) ]
#     return ls

# index_list = []
# for feature in df_prepared.columns:
#     if pd.api.types.is_numeric_dtype(df_prepared[feature]):
#         index_list.extend(outliers1(df_prepared, feature))

# def remove(df, ls):
#     ls = sorted(set(ls))
#     df = df.drop(ls)
#     return df

# print(f"There are {len(set(index_list))} outliers")
# print(f"DataFrame Shape Before Removing Outliers {df_prepared.shape}")
# df_cleaned_after_outliers = remove(df_prepared, index_list)
# print(f"DataFrame Shape After  Removing Outliers {df_cleaned_after_outliers.shape}")


<span style="color:cyan">

##### 3.3. Outliers - To remove instances
##### Analysis

To-Do

Please refer to the outliers section report in the Data Understanding section.

</span>

In [27]:
# 3.4. Consistency - Invalid formats / Ranges

df_cleaned = df_clean_outliers.copy()

# Convert CRASH_DATE to datetime
df_cleaned['CRASH_DATE'] = pd.to_datetime(df_cleaned['CRASH_DATE'])

# Extract components from CRASH_DATE
df_cleaned['YEAR'] = df_cleaned['CRASH_DATE'].dt.year

# 4. CRISP-DM: Modeling

**Section Overview**

4.1. Import relevant ML libs for Modeling.<br>
4.2. Formaluize a Numerical Measure for the Target Variable(s).<br>
4.3. Undersampling - To fix over-representation in the dataset (unbalanced data).<br>
4.4. Modeling with Random Forest.<br>
4.5. Modeling with Logestic Regression.<br>
4.6. Modeling with DNN.<br>
4.7. Modeling with KNN.<br>

In [24]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

# 5. CRISP-DM: Evaluation

To-Do.

# 6. CRISP-DM: Deployment

To-Do.